-sandbox
# Exercise User Defined Functions
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Instructions

In this exercise, we're doing ETL. Instead of using built-in functions, use UDFs to complete the exercise. 

As a reminder, that file contains data about people, including:


* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL"). 
* The Social Security numbers aren't consistent, either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. (The salaries will match, as well,
and the Social Security Numbers *would* match, if they were somehow put in the same format).

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lower-case, you haven't met this requirement.
* Write the result as a Parquet file, as designated by *destFile*.
* The final Parquet "file" must contain 8 part files (8 files ending in ".parquet").

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** The initial dataset contains 103,000 records.<br/>
The de-duplicated result haves 100,000 records.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%pip install tensorflow-cpu==2.4.*

Python interpreter will be restarted.
Collecting tensorflow-cpu==2.4.*
 Downloading tensorflow_cpu-2.4.3-cp38-cp38-manylinux2010_x86_64.whl (144.4 MB)
Collecting google-pasta~=0.2
 Downloading google_pasta-0.2.0-py3-none-any.whl (57 kB)
Requirement already satisfied: protobuf>=3.9.2 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (3.17.0)
Requirement already satisfied: wheel~=0.35 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (0.35.1)
Collecting h5py~=2.10.0
 Downloading h5py-2.10.0-cp38-cp38-manylinux1_x86_64.whl (2.9 MB)
Collecting astunparse~=1.6.3
 Downloading astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Collecting typing-extensions~=3.7.4
 Downloading typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
Collecting termcolor~=1.1.0
 Downloading termcolor-1.1.0.tar.gz (3.9 kB)
Collecting tensorboard~=2.4
 Downloading tensorboard-2.6.0-py3-none-any.whl (5.6 MB)
Collecting flatbuffers~=1.12.0
 Downloading flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
Collecting wrapt~=1.12.1
 Downloading wrapt-1.12.1.tar.gz (27 kB)
Collecting absl-py~=0.10
 Downloading absl_py-0.13.0-py3-none-any.whl (132 kB)
Collecting gast==0.3.3
 Downloading gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
Collecting keras-preprocessing~=1.1.2
 Downloading Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
Collecting grpcio~=1.32.0
 Downloading grpcio-1.32.0-cp38-cp38-manylinux2014_x86_64.whl (3.8 MB)
Collecting tensorflow-estimator<2.5.0,>=2.4.0
 Downloading tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
Requirement already satisfied: six~=1.15.0 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.15.0)
Requirement already satisfied: numpy~=1.19.2 in /databricks/python3/lib/python3.8/site-packages (from tensorflow-cpu==2.4.*) (1.19.2)
Collecting opt-einsum~=3.3.0
 Downloading opt_einsum-3.3.0-py3-none-any.whl (65 kB)
Collecting google-auth<2,>=1.6.3
 Downloading google_auth-1.35.0-py2.py3-none-any.whl (152 kB)
Collecting werkzeug>=0.11.15
 Downloading Werkzeug-2.0.1-py3-none-any.whl (288 kB)
Requirement already satisfied: requests<3,>=2.21.0 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (2.24.0)
Collecting markdown>=2.6.8
 Downloading Markdown-3.3.4-py3-none-any.whl (97 kB)
Collecting tensorboard-data-server<0.7.0,>=0.6.0
 Downloading tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
Collecting google-auth-oauthlib<0.5,>=0.4.1
 Downloading google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
Collecting tensorboard-plugin-wit>=1.6.0
 Downloading tensorboard_plugin_wit-1.8.0-py3-none-any.whl (781 kB)
Requirement already satisfied: setuptools>=41.0.0 in /databricks/python3/lib/python3.8/site-packages (from tensorboard~=2.4->tensorflow-cpu==2.4.*) (50.3.1)
Collecting rsa<5,>=3.1.4; python_version >= "3.6"
 Downloading rsa-4.7.2-py3-none-any.whl (34 kB)
Collecting pyasn1-modules>=0.2.1
 Downloading pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
Collecting cachetools<5.0,>=2.0.0
 Downloading cachetools-4.2.2-py3-none-any.whl (11 kB)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.21.0->tensorboard~=2.4->tensorflow-cpu==2.4.*) (2020.12.5)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.21.0->tensorboard~=2.4->tensorflow-cpu==2.4.*) (3.0.4)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.21.0->tensorboard~=2.4->tensorflow-cpu==2.4.*) (2.10)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.21.0->tensorboard~=2.4->tensorflow-cpu==2.4.*) (1.25.11)
Collecting requests-oauthlib>=0.7.0
 Downloading requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
Collecting pyasn1>=0.1.3
 Downloadi

In [0]:
%run "./Includes/Classroom-Setup"

Preparing the learning environment...

Preparing the Scala environment...

Preparing the Scala environment...

Preparing the Python environment...

Preparing the Python environment...

...

Initializing Databricks Academy's services for generating dynamic data...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from wasbs://training@dbtraineastus.blob.core.windows.net/

Cleaning up the learning environment...no actions taken.

Cleaning up the learning environment...no actions taken.

The following functions were defined for you: 
 waitForMLflow 
 ( 
 
 ) 
 MLflow is an open source platform to manage the ML lifecycle, including experimentation, reproducibility and deployment 
 
 display_run_uri 
 ( 
 experiment_id, run_id 
 ) 
 Experiment and run ids 
 
 plot_confusion_matrix 
 ( 
 y_true, y_pred, classes, title=None, cmap=plt.cm.Blues 
 ) 
 Confusion matrix 
 The following variables were defined for you: 
 username : vsekar@cloudseclab2.com 
 No additional information was provided. 
 
 workingDir : dbfs:/user/vsekar@cloudseclab2.com/deep_learning/2_exercise_user_defined_functions_pil 
 No additional information was provided. 
 
 working_path : /dbfs/user/vsekar@cloudseclab2.com/deep_learning/2_exercise_user_defined_functions_pil 
 This is working directory. 
 
 ml_working_path : /dbfs/ml/vsekar@cloudseclab2.com 
 This is ML working directory. 
 All done!

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Hints

* Use the <a href="http://spark.apache.org/docs/latest/api/python/index.html" target="_blank">API docs</a>. Specifically, you might find 
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame" target="_blank">DataFrame</a> and
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">functions</a> to be helpful.
* It's helpful to look at the file first, so you can check the format. `dbutils.fs.head()` (or just `%fs head`) is a big help here.

In [0]:
# TODO

sourceFile = "dbfs:/mnt/training/dataframes/people-with-dups.txt"
destFile = userhome + "/people.parquet"

# In case it already exists
dbutils.fs.rm(destFile, True)

Out[14]: False

##![Spark Logo Tiny](https://s3-us-west-2.amazonaws.com/curriculum-release/images/105/logo_spark_tiny.png) Validate Your Answer

At the bare minimum, we can verify that you wrote the parquet file out to **destFile** and that you have the right number of records.

Running the following cell to confirm your result:

In [0]:
partFiles = len(list(filter(lambda f: f.path.endswith(".parquet"), dbutils.fs.ls(destFile))))

finalDF = spark.read.parquet(destFile)
finalCount = finalDF.count()

clearYourResults()
validateYourAnswer("01 Parquet File Exists", 1276280174, partFiles)
validateYourAnswer("02 Expected 100000 Records", 972882115, finalCount)
summarizeYourResults()

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-4349241570437324> in <module> 
 ----> 1 partFiles = len ( list ( filter ( lambda f : f . path . endswith ( ".parquet" ) , dbutils . fs . ls ( destFile ) ) ) ) 
 2 
 3 finalDF = spark . read . parquet ( destFile ) 
 4 finalCount = finalDF . count ( ) 
 5 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 328 exc . __context__ = None 
 329 exc . __cause__ = None 
 --> 330 raise exc
 331 return f_with_exception_handling
 332 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: java.io.FileNotFoundException: File /791337650924823/user/vsekar@cloudseclab2.com/people.parquet does not exist.
	at shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem.listStatus(NativeAzureFileSystem.java:2387)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:95)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:66)
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$1(DatabricksFileSystemV2.scala:92)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:434)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:455)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:279)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:271)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:455)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:415)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:341)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.recordOperation(DatabricksFileSystemV2.scala:455)
	at com.databricks.backend.daemon.data.client.DBFSV2.listStatus(DatabricksFileSystemV2.scala:92)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:150)
	at com.databricks.backend.daemon.dbutils.FSUtils$.$anonfun$ls$1(DBUtilsCore.scala:154)
	at com.databricks.backend.daemon.dbutils.FSUtils$.withFsSafetyCheck(DBUtilsCore.scala:91)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:153)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)